# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [174]:
%%time
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
pd.set_option("display.max_rows", 500, "display.max_columns", 500)
from nltk.tokenize import word_tokenize
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.probability import FreqDist
from nltk import NaiveBayesClassifier
from sklearn.model_selection import train_test_split

Wall time: 144 ms
Parser   : 222 ms


In [175]:
data = pd.read_csv('..\..\..\IronHack\8.04 Natural Language Processing\Sentiment140.csv')

In [176]:
data.columns

Index(['target', 'ids', 'date', 'flag', 'user', 'text'], dtype='object')

In [177]:
data.columns = ['target','id','date','flag','user','text']
data.columns

Index(['target', 'id', 'date', 'flag', 'user', 'text'], dtype='object')

In [178]:
# data_sample = data.head(10000) due to the order, my accuracy score is 1 so I will use sample instead
data_sample = data.sample(n = 20000)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [179]:
def clean_up(s):
    
    return re.sub('[^a-z]', ' ', re.sub('http[s]?://[/\w\.]*', '', s.lower())).strip()

In [180]:
clean = data.text.apply(clean_up)

In [181]:
def tokenize(s):
    s = word_tokenize(s)
    s = [word for word in s if word.isalnum()]
    return s

In [182]:
tokenized = clean.apply(tokenize)

In [183]:
def stem_and_lemmatize(l):
    ps = PorterStemmer()
    stemmed = [ps.stem(w) for w in l]
    lemmatizer = WordNetLemmatizer() 
    lemmatized = [lemmatizer.lemmatize(word) for word in stemmed]
    return lemmatized

In [184]:
stemlem = tokenized.apply(stem_and_lemmatize)

In [185]:
def remove_stopwords(l):
    l = [word for word in l if not word in stopwords.words('english')]
    return l

In [186]:
%%time
stopwords = stemlem.apply(remove_stopwords)
stopwords.head()

Wall time: 3h 25min 26s
Parser   : 149 ms


0    [switchfoot, awww, bummer, shoulda, got, david...
1    [upset, updat, hi, facebook, text, might, cri,...
2    [kenichan, dive, mani, time, ball, manag, save...
3               [whole, bodi, feel, itchi, like, fire]
4      [nationwideclass, behav, mad, whi, becaus, see]
Name: text, dtype: object

In [187]:
data['text_processed'] = stopwords
data

,target,id,date,flag,user,text,text_processed
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","[switchfoot, awww, bummer, shoulda, got, david..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,"[upset, updat, hi, facebook, text, might, cri,..."
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,"[kenichan, dive, mani, time, ball, manag, save..."
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,"[whole, bodi, feel, itchi, like, fire]"
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","[nationwideclass, behav, mad, whi, becaus, see]"
...,...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...,"[woke, school, best, feel, ever]"
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,"[thewdb, com, veri, cool, hear, old, walt, int..."
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,"[readi, mojo, makeov, ask, detail]"
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,"[happi, th, birthday, boo, alll, time, tupac, ..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [188]:
def flatten(input_list):
    return [item for sublist in input_list for item in sublist]

In [189]:
processed = flatten(data['text_processed'])
freqdist = FreqDist(processed).most_common(5000)
pd.DataFrame(freqdist).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,4750,4751,4752,4753,4754,4755,4756,4757,4758,4759,4760,4761,4762,4763,4764,4765,4766,4767,4768,4769,4770,4771,4772,4773,4774,4775,4776,4777,4778,4779,4780,4781,4782,4783,4784,4785,4786,4787,4788,4789,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800,4801,4802,4803,4804,4805,4806,4807,4808,4809,4810,4811,4812,4813,4814,4815,4816,4817,4818,4819,4820,4821,4822,4823,4824,4825,4826,4827,4828,4829,4830,4831,4832,4833,4834,4835,4836,4837,4838,4839,4840,4841,4842,4843,4844,4845,4846,4847,4848,4849,4850,4851,4852,4853,4854,4855,4856,4857,4858,4859,4860,4861,4862,4863,4864,4865,4866,4867,4868,4869,4870,4871,4872,4873,4874,4875,4876,4877,4878,4879,4880,4881,4882,4883,4884,4885,4886,4887,4888,4889,4890,4891,4892,4893,4894,4895,4896,4897,4898,4899,4900,4901,4902,4903,4904,4905,4906,4907,4908,4909,4910,4911,4912,4913,4914,4915,4916,4917,4918,4919,4920,4921,4922,4923,4924,4925,4926,4927,4928,4929,4930,4931,4932,4933,4934,4935,4936,4937,4938,4939,4940,4941,4942,4943,4944,4945,4946,4947,4948,4949,4950,4951,4952,4953,4954,4955,4956,4957,4958,4959,4960,4961,4962,4963,4964,4965,4966,4967,4968,4969,4970,4971,4972,4973,4974,4975,4976,4977,4978,4979,4980,4981,4982,4983,4984,4985,4986,4987,4988,4989,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,go,get,day,wa,thi,good,work,like,love,quot,u,got,today,time,lol,thank,miss,one,back,want,know,see,feel,think,im,realli,amp,night,hope,watch,still,need,make,well,new,na,home,oh,look,ha,come,much,last,twitter,morn,tomorrow,wish,great,wait,sleep,haha,sad,fun,whi,week,tri,onli,right,follow,happi,bad,would,veri,thing,sorri,tonight,friend,say,way,hi,take,gon,nice,though,better,hate,even,yeah,tweet,bed,could,start,school,hour,peopl,show,guy,play,weekend,hey,ye,final,awesom,next,let,lt,use,x,dont,never,soon,cant,tire,long,rain,plea,littl,life,first,everyon,wan,year,movi,best,girl,sick,ok,find,suck,call,ani,sure,done,help,bore,head,alway,talk,keep,alreadi,cool,live,anoth,lot,someth,eat,phone,man,n,leav,read,hurt,readi,made,yay,enjoy,becaus,song,hous,yet,ur,went,ever,befor,sound,thought,pretti,mayb,excit,away,summer,game,omg,finish,amaz,guess,old,tell,b,damn,mean,earli,listen,someon,babi,check,bit,lost,left,give,w,big,hot,end,wow,parti,late,noth,hear,ya,r,actual,pic,birthday,glad,happen,sun,stop,also,weather,later,doe,two,p,mom,th,wonder,stuff,ugh,put,ta,saw,run,exam,music,god,fuck,stay,might,hard,world,car,kid,yesterday,said,meet,sinc,job,beauti,post,updat,com,sunday,friday,monday,around,video,mani,seem,cold,cri,luck,must,found,boy,book,move,die,aww,busi,...,inaperfectworld,northern,stain,stiff,platform,rach,canal,wp,yung,bookmark,awwwwww,methink,lolli,pinch,pacif,spoken,estat,independ,ford,tart,crown,infam,andyhurleyday,sod,genuin,goat,woe,electron,overload,spaghetti,dollhous,slower,creek,chai,kerri,sketch,dial,tackl,hashtag,monthli,tiff,munch,scout,reciev,loll,outing,treasur,hawk,agenda,defo,financ,glove,dec,antonio,greatli,sparkl,auction,compet,ada,checkin,psycholog,understood,gov,israel,transit,tale,calcul,assassin,summit,cobra,kong,blu,jungl,xoxoxo,hunger,imax,dw,lovey,rustyrocket,mk,interfac,western,kudo,tc,lola,twitterfox,l

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [190]:
freq_df = pd.DataFrame(freqdist).T
freq_df.iloc[0]

0           go
1          get
2          day
3           wa
4          thi
         ...  
4995    vacuum
4996    shelbi
4997    memphi
4998     humbl
4999     shawn
Name: 0, Length: 5000, dtype: object

In [191]:
freq_df.columns = freq_df.iloc[0]

In [192]:
freq_df

,go,get,day,wa,thi,good,work,like,love,quot,u,got,today,time,lol,thank,miss,one,back,want,know,see,feel,think,im,realli,amp,night,hope,watch,still,need,make,well,new,na,home,oh,look,ha,come,much,last,twitter,morn,tomorrow,wish,great,wait,sleep,haha,sad,fun,whi,week,tri,onli,right,follow,happi,bad,would,veri,thing,sorri,tonight,friend,say,way,hi,take,gon,nice,though,better,hate,even,yeah,tweet,bed,could,start,school,hour,peopl,show,guy,play,weekend,hey,ye,final,awesom,next,let,lt,use,x,dont,never,soon,cant,tire,long,rain,plea,littl,life,first,everyon,wan,year,movi,best,girl,sick,ok,find,suck,call,ani,sure,done,help,bore,head,alway,talk,keep,alreadi,cool,live,anoth,lot,someth,eat,phone,man,n,leav,read,hurt,readi,made,yay,enjoy,becaus,song,hous,yet,ur,went,ever,befor,sound,thought,pretti,mayb,excit,away,summer,game,omg,finish,amaz,guess,old,tell,b,damn,mean,earli,listen,someon,babi,check,bit,lost,left,give,w,big,hot,end,wow,parti,late,noth,hear,ya,r,actual,pic,birthday,glad,happen,sun,stop,also,weather,later,doe,two,p,mom,th,wonder,stuff,ugh,put,ta,saw,run,exam,music,god,fuck,stay,might,hard,world,car,kid,yesterday,said,meet,sinc,job,beauti,post,updat,com,sunday,friday,monday,around,video,mani,seem,cold,cri,luck,must,found,boy,book,move,die,aww,busi,...,inaperfectworld,northern,stain,stiff,platform,rach,canal,wp,yung,bookmark,awwwwww,methink,lolli,pinch,pacif,spoken,estat,independ,ford,tart,crown,infam,andyhurleyday,sod,genuin,goat,woe,electron,overload,spaghetti,dollhous,slower,creek,chai,kerri,sketch,dial,tackl,hashtag,monthli,tiff,munch,scout,reciev,loll,outing,treasur,hawk,agenda,defo,financ,glove,dec,antonio,greatli,sparkl,auction,compet,ada,checkin,psycholog,understood,gov,israel,transit,tale,calcul,assassin,summit,cobra,kong,blu,jungl,xoxoxo,hunger,imax,dw,lovey,rustyrocket,mk,interfac,western,kudo,tc,lola,twitterfox,loooong,cleaner,sprinkl,egypt,ralli,samantha,hump,eagl,frankiethesat,bowwow,por,spit,alik,buffet,fucker,corrupt,billion,wal,utterli,wilson,strep,nm,helen,kidnap,nowaday,led,bri,ghostbust,aloud,freezer,unknown,mbp,buffi,adapt,birdi,documentari,ment,fearnecotton,autumn,survey,benjamin,knife,syrup,britt,espn,delish,mash,rail,inner,gotcha,seal,appeal,incid,ijustin,ick,snooz,wrestl,ontd,pedi,mariah,jill,sandi,hidden,heehe,freshman,jewelri,ken,editor,dp,witch,overtim,fascin,debut,flame,garbag,restless,saint,dash,fluffi,ky,yaa,nicknam,lesbian,mgiraudoffici,agenc,izzi,buse,gordon,firefli,mu,juss,melodi,timelin,zack,sarcasm,belgium,od,nov,ro,elev,jasonbradburi,stereo,chipotl,tether,gulpanag,mushroom,spice,heidimontag,brake,chap,wardrob,trampolin,insult,limp,repres,fighter,nacho,yayyyi,complaint,ramen,raspberri,dt,lori,whiskey,oregon,olivia,ting,sumthin,tin,lv,newbi,invis,barri,colin,endless,tow,kardashian,pam,jar,nvm,ui,harper,kiwi,captur,hamburg,haveyouev,throughout,connor,flirt,mat,compat,nike,qu,tomo,safeti,layer,foundat,grandfath,badg,vacuum,shelbi,memphi,humbl,shawn
0,go,get,day,wa,thi,good,work,like,love,quot,u,got,today,time,lol,thank,miss,one,back,want,know,see,feel,think,im,realli,amp,night,hope,watch,still,need,make,well,new,na,home,oh,look,ha,come,much,last,twitter,morn,tomorrow,wish,great,wait,sleep,haha,sad,fun,whi,week,tri,onli,right,follow,happi,bad,would,veri,thing,sorri,tonight,friend,say,way,hi,take,gon,nice,though,better,hate,even,yeah,tweet,bed,could,start,school,hour,peopl,show,guy,play,weekend,hey,ye,final,awesom,next,let,lt,use,x,dont,never,soon,cant,tire,long,rain,plea,littl,life,first,everyon,wan,year,movi,best,girl,sick,ok,find,suck,call,ani,sure,done,help,bore,head,alway,talk,keep,alreadi,cool,live,anoth,lot,someth,eat,phone,man,n,leav,read,hurt,readi,made,yay,enjoy,becaus,song,hous,yet,ur,went,ever,befor,sound,thought,pretti,mayb,excit,away,summer,game,omg,finish,amaz,guess,old,tell,b,damn,mean,earli,listen,someon,babi,check,bit,lost,left,give,w,big,hot,end,wow,parti,late,noth,hear,ya,r,actual,pic,birthday,glad,happen,sun,stop,also,weather,later,doe,two,p,mom,th,wonder,stuff,ugh,put,ta,saw,r

In [193]:
freq_df.drop([0], axis = 0, inplace = True) 

In [194]:
freq_df

,go,get,day,wa,thi,good,work,like,love,quot,u,got,today,time,lol,thank,miss,one,back,want,know,see,feel,think,im,realli,amp,night,hope,watch,still,need,make,well,new,na,home,oh,look,ha,come,much,last,twitter,morn,tomorrow,wish,great,wait,sleep,haha,sad,fun,whi,week,tri,onli,right,follow,happi,bad,would,veri,thing,sorri,tonight,friend,say,way,hi,take,gon,nice,though,better,hate,even,yeah,tweet,bed,could,start,school,hour,peopl,show,guy,play,weekend,hey,ye,final,awesom,next,let,lt,use,x,dont,never,soon,cant,tire,long,rain,plea,littl,life,first,everyon,wan,year,movi,best,girl,sick,ok,find,suck,call,ani,sure,done,help,bore,head,alway,talk,keep,alreadi,cool,live,anoth,lot,someth,eat,phone,man,n,leav,read,hurt,readi,made,yay,enjoy,becaus,song,hous,yet,ur,went,ever,befor,sound,thought,pretti,mayb,excit,away,summer,game,omg,finish,amaz,guess,old,tell,b,damn,mean,earli,listen,someon,babi,check,bit,lost,left,give,w,big,hot,end,wow,parti,late,noth,hear,ya,r,actual,pic,birthday,glad,happen,sun,stop,also,weather,later,doe,two,p,mom,th,wonder,stuff,ugh,put,ta,saw,run,exam,music,god,fuck,stay,might,hard,world,car,kid,yesterday,said,meet,sinc,job,beauti,post,updat,com,sunday,friday,monday,around,video,mani,seem,cold,cri,luck,must,found,boy,book,move,die,aww,busi,...,inaperfectworld,northern,stain,stiff,platform,rach,canal,wp,yung,bookmark,awwwwww,methink,lolli,pinch,pacif,spoken,estat,independ,ford,tart,crown,infam,andyhurleyday,sod,genuin,goat,woe,electron,overload,spaghetti,dollhous,slower,creek,chai,kerri,sketch,dial,tackl,hashtag,monthli,tiff,munch,scout,reciev,loll,outing,treasur,hawk,agenda,defo,financ,glove,dec,antonio,greatli,sparkl,auction,compet,ada,checkin,psycholog,understood,gov,israel,transit,tale,calcul,assassin,summit,cobra,kong,blu,jungl,xoxoxo,hunger,imax,dw,lovey,rustyrocket,mk,interfac,western,kudo,tc,lola,twitterfox,loooong,cleaner,sprinkl,egypt,ralli,samantha,hump,eagl,frankiethesat,bowwow,por,spit,alik,buffet,fucker,corrupt,billion,wal,utterli,wilson,strep,nm,helen,kidnap,nowaday,led,bri,ghostbust,aloud,freezer,unknown,mbp,buffi,adapt,birdi,documentari,ment,fearnecotton,autumn,survey,benjamin,knife,syrup,britt,espn,delish,mash,rail,inner,gotcha,seal,appeal,incid,ijustin,ick,snooz,wrestl,ontd,pedi,mariah,jill,sandi,hidden,heehe,freshman,jewelri,ken,editor,dp,witch,overtim,fascin,debut,flame,garbag,restless,saint,dash,fluffi,ky,yaa,nicknam,lesbian,mgiraudoffici,agenc,izzi,buse,gordon,firefli,mu,juss,melodi,timelin,zack,sarcasm,belgium,od,nov,ro,elev,jasonbradburi,stereo,chipotl,tether,gulpanag,mushroom,spice,heidimontag,brake,chap,wardrob,trampolin,insult,limp,repres,fighter,nacho,yayyyi,complaint,ramen,raspberri,dt,lori,whiskey,oregon,olivia,ting,sumthin,tin,lv,newbi,invis,barri,colin,endless,tow,kardashian,pam,jar,nvm,ui,harper,kiwi,captur,hamburg,haveyouev,throughout,connor,flirt,mat,compat,nike,qu,tomo,safeti,layer,foundat,grandfath,badg,vacuum,shelbi,memphi,humbl,shawn
1,138781,110866,109408,105045,94815,92591,87886,83852,83121,73411,71368,71120,68717,66420,59484,59443,58028,57619,57419,57355,54977,51387,51268,51213,50791,50084,48852,45829,44956,43648,43625,43450,43222,42968,42482,42182,40855,40052,39761,39531,39137,37068,36275,36258,35732,34340,34043,33695,32605,32475,31543,30046,28735,28539,28343,28271,28198,28123,28065,27768,27433,27335,26913,26533,26431,26183,26074,25671,25219,24996,24744,24101,24101,24017,23267,23033,22870,22520,22491,22435,21930,21804,21092,21044,20990,20795,19999,19705,19619,19153,19034,18995,18814,18677,18647,18609,18518,18262,18247,17957,17857,17689,17379,17257,17250,17241,17065,16892,16834,16803,16763,16752,16645,16531,16466,16396,16266,15813,15700,15669,15528,15472,15371,15343,15315,15263,14989,14973,14794,14780,14764,14669,14599,14590,14488,14377,14351,14292,14245,14187,14166,14148,14003,13862,13814,13746,13632,13528,13438,13402,13325,13308,13162,13071,12847,12785,12750,12686,12439,12326,12311,12303,12269,12265,12265,12197,12196,12171,12146,12006,11903,11846,11841,11744,11701,11590,11552,

In [195]:
def find_features(document):
    words = set(document)
    return {w:(w in words) for w in freq_df.columns}

In [196]:
documents = list(zip(data['text_processed'], np.where(data['target'] == 4, True, False)))
#documents

In [ ]:
%%time
featuresets = [(find_features(words), category) for (words, category) in documents]

In [198]:
#featuresets

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [199]:
train_set, test_set = featuresets[:500], featuresets[500:]

In [200]:
%%time
classifier = nltk.NaiveBayesClassifier.train(train_set)

Wall time: 40.4 s
Parser   : 227 ms


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [201]:
%%time
nltk.classify.accuracy(classifier, test_set)

Wall time: 8min 21s
Parser   : 234 ms


0.6273846153846154

In [202]:
%%time
classifier.show_most_informative_features(15)

Most Informative Features
                    miss = True            False : True   =      6.6 : 1.0
                   still = True            False : True   =      6.3 : 1.0
                    haha = True             True : False  =      5.0 : 1.0
                       c = True             True : False  =      4.3 : 1.0
                    damn = True            False : True   =      4.3 : 1.0
                    girl = True             True : False  =      4.3 : 1.0
                  wonder = True             True : False  =      4.3 : 1.0
                    wish = True            False : True   =      3.9 : 1.0
                     fun = True             True : False  =      3.8 : 1.0
                     hey = True             True : False  =      3.8 : 1.0
                     tri = True            False : True   =      3.8 : 1.0
                   anoth = True             True : False  =      3.7 : 1.0
                     buy = True            False : True   =      3.7 : 1.0

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [142]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [143]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [144]:
# your code here